In [1]:
import os
import dotenv

from sqlalchemy import Column, Integer, create_engine, Text
from sqlalchemy.orm import declarative_base, Session
from tidb_vector.sqlalchemy import VectorType

from sqlalchemy import URL

dotenv.load_dotenv()

True

In [2]:
# Step 1: Connect to TiDB using SQLAlchemy.
tidb_connection_string = URL(
    "mysql+pymysql",
    username=os.environ['TIDB_USERNAME'],
    password=os.environ['TIDB_PASSWORD'],
    host=os.environ['TIDB_HOST'],
    port=4000,
    database="test",
    query={"ssl_verify_cert": True, "ssl_verify_identity": True},
)
engine = create_engine(tidb_connection_string)

# Step 2: Define a table with a vector column.
Base = declarative_base()

# Issue : https://stackoverflow.com/questions/454854/no-module-named-mysqldb

In [3]:
class Document(Base):
    __tablename__ = 'sqlalchemy_demo_documents'
    id = Column(Integer, primary_key=True)
    content = Column(Text)
    embedding = Column(VectorType(1536))

In [4]:
class DocumentWithIndex(Base):
    __tablename__ = 'sqlalchemy_demo_documents_with_index'
    id = Column(Integer, primary_key=True)
    content = Column(Text)
    embedding = Column(VectorType(1536), comment="hnsw(distance=cosine)")


In [5]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [6]:
EXAMPLE_1 = """
To become a leader, you need to demonstrate several key behaviors. Here's a list of actions and competencies that are highly related to leadership:

1. Give clear directions to team members and ensure they are understood.
2. Coordinate team activities.
3. Set clear, measurable goals and objectives for team members.
4. Adapt your approach to the individual or situation.
5. Keep team members focused on important objectives.
6. Create a vision, bring meaning, be strategic, inspire, involve, align, communicate, connect, motivate (these are some of the roles a leader plays).
7. Define goals, plan, organize, coordinate, forecast, track progress, review (these are some of the tasks a manager does).
8. Impart specific domain knowledge, share functional expertise, teach technical skills (these are some of the roles a mentor plays).
9. Guide, develop, encourage, stimulate, engage, confront, challenge, advise (these are some of the roles a coach plays).
"""

EXAMPLE_2 = """
Boosting Confidence: Quick Tips
1. Know Your Strengths:
2. Identify what you’re good at and celebrate your successes, no matter how small.
3. Set Achievable Goals:
4. Break tasks into small, manageable steps and track your progress.
5. Practice Positive Self-Talk:
6. Replace negative thoughts with positive affirmations to reinforce self-belief.
7. Challenge yourself with new experiences to build resilience and self-assurance.
"""

In [7]:
# Get text and embeddings
EMBEDDING_MODEL_ENDPOINT = "https://api.openai.com/v1/embeddings"
import os
import openai
from dotenv import load_dotenv
load_dotenv()


# Load environment variables
OPENAI_ORG = os.getenv('OPENAI_ORG')
OPENAI_APIKEY = os.getenv('OPENAI_APIKEY')

openai.organization = OPENAI_ORG
openai.api_key = OPENAI_APIKEY


openai_client = openai.OpenAI(api_key=OPENAI_APIKEY)
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

In [8]:
with Session(engine) as session:
    session.add(Document(content=EXAMPLE_1, embedding=get_embedding(EXAMPLE_1)))
    session.add(Document(content=EXAMPLE_2, embedding=get_embedding(EXAMPLE_2)))
    session.commit()


In [9]:
user_query = "How to become a leader?"
user_query_embedding = get_embedding(user_query)

In [11]:
# Step 4: Get the 3-nearest neighbor documents.
print('Get 3-nearest neighbor documents:')
with Session(engine) as session:
    distance = Document.embedding.cosine_distance(user_query_embedding).label('distance')
    results = session.query(Document, distance).order_by(distance).limit(1).all()

    for doc, distance in results:
        print(f'  - distance: {distance}\n'
              f'    document: {doc.content}')

Get 3-nearest neighbor documents:
  - distance: 0.1365278946070263
    document: 
To become a leader, you need to demonstrate several key behaviors. Here's a list of actions and competencies that are highly related to leadership:

1. Give clear directions to team members and ensure they are understood.
2. Coordinate team activities.
3. Set clear, measurable goals and objectives for team members.
4. Adapt your approach to the individual or situation.
5. Keep team members focused on important objectives.
6. Create a vision, bring meaning, be strategic, inspire, involve, align, communicate, connect, motivate (these are some of the roles a leader plays).
7. Define goals, plan, organize, coordinate, forecast, track progress, review (these are some of the tasks a manager does).
8. Impart specific domain knowledge, share functional expertise, teach technical skills (these are some of the roles a mentor plays).
9. Guide, develop, encourage, stimulate, engage, confront, challenge, advise (these

In [12]:
# Step 5: Get documents within a certain distance.
print('Get documents within a certain distance:')
with (Session(engine) as session):
    distance = Document.embedding.cosine_distance(user_query_embedding).label('distance')
    results = session.query(
        Document, distance
    ).filter(distance < 0.2).order_by(distance).limit(1).all()

    for doc, distance in results:
        print(f'  - distance: {distance}\n'
              f'    document: {doc.content}')


Get documents within a certain distance:
  - distance: 0.1365278946070263
    document: 
To become a leader, you need to demonstrate several key behaviors. Here's a list of actions and competencies that are highly related to leadership:

1. Give clear directions to team members and ensure they are understood.
2. Coordinate team activities.
3. Set clear, measurable goals and objectives for team members.
4. Adapt your approach to the individual or situation.
5. Keep team members focused on important objectives.
6. Create a vision, bring meaning, be strategic, inspire, involve, align, communicate, connect, motivate (these are some of the roles a leader plays).
7. Define goals, plan, organize, coordinate, forecast, track progress, review (these are some of the tasks a manager does).
8. Impart specific domain knowledge, share functional expertise, teach technical skills (these are some of the roles a mentor plays).
9. Guide, develop, encourage, stimulate, engage, confront, challenge, advise